In [216]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

# Initialize strategies from Axelrod's first tournament
strategies = [s() for s in axl.axelrod_first_strategies]
number_of_strategies = len(strategies)

# Initialize population proportions (equal distribution)
population = np.full(number_of_strategies, 1 / number_of_strategies)

# Function to play a round of IPD and return payoffs
def play_round(player1, player2, turns=5):
    # Create a match between player1 and player2
    match = axl.Match([player1, player2], turns=turns)
    # Play the match
    match.play()
    # Return the final scores
    return match.final_score()

# Function to calculate fitness
def calculate_fitness():
    fitness = np.zeros(number_of_strategies)
    for i, player1 in enumerate(strategies):
        for j, player2 in enumerate(strategies):
            if i != j:
                payoff = play_round(player1, player2)
                fitness[i] += sum(payoff) * population[j]
    return fitness

# Evolutionary dynamics (e.g., replicator dynamics)
def evolve_population(fitness):
    total_fitness = np.sum(fitness)
    return population * fitness / total_fitness

# Simulation parameters
num_generations = 100
history = []

# Simulation loop
for _ in range(num_generations):
    history.append(population.copy())
    fitness = calculate_fitness()
    population = evolve_population(fitness)

# Plotting results
history = np.array(history)

# Convert history to DataFrame for easier plotting
history_df = pd.DataFrame(history, columns=[str(player) for player in strategies])
history_df['Generation'] = history_df.index

# Normalize the data if not already normalized
history_df_normalized = history_df.copy()
for col in [str(player) for player in strategies]:
    history_df_normalized[col] = history_df[col] / history_df[[str(player) for player in strategies]].sum(axis=1)

# Prepare data for Plotly
history_long_df = history_df_normalized.melt(id_vars='Generation', 
                                  value_vars=[str(s) for s in strategies],
                                  var_name='Strategy', value_name='Population Proportion')

def calculate_cooperation_rating(strategy_classes):
    cooperation_ratings = {}
    for strategy_class in strategy_classes:
        player = strategy_class()  # Instantiate the strategy
        opponent = axl.Cooperator()  # A strategy that always cooperates
        match = axl.Match([player, opponent], turns=200)
        match.play()
        # Calculate cooperation rating
        cooperation_rating = player.cooperations / match.turns
        strategy_name = str(strategy_class())  # Standardize the name
        cooperation_ratings[strategy_name] = cooperation_rating
    return cooperation_ratings

cooperation_ratings = calculate_cooperation_rating(axl.axelrod_first_strategies)

# Use cooperation_ratings dict to add cooperation ratings to DataFrame
history_long_df['Cooperation Rating'] = history_long_df['Strategy'].map(cooperation_ratings)


In [ ]:
# Describe and document the above simulation 
# (e.g., what is the purpose of the simulation, what are the parameters, etc.)
"""
The purpose of the simulation is to show the evolution of the population of strategies
over time. The simulation takes the population of strategies and plays them against each
other. The fitness of each strategy is calculated based on the payoff from playing against
each other strategy. The population then evolves based on the fitness of each strategy.
The simulation is run for 100 generations.
"""

# How is fitness calculated 
# (e.g., what is the payoff matrix, how many rounds are played, etc.)
"""
The fitness is calculated by playing each strategy against each other strategy. The payoff
matrix is the standard payoff matrix for the iterated prisoner's dilemma. The number of rounds
played is 5.
"""

# What is the evolutionary dynamics (e.g., replicator dynamics, etc.)
"""
The evolutionary dynamics is replicator dynamics.
"""


In [209]:
history_long_df.head()

,Generation,Strategy,Population Proportion,Cooperation Rating
0,0,Tit For Tat,0.066667,1.0
1,1,Tit For Tat,0.068739,1.0
2,2,Tit For Tat,0.072162,1.0
3,3,Tit For Tat,0.075509,1.0
4,4,Tit For Tat,0.076708,1.0


In [217]:
sort_order = history_long_df['Cooperation Rating'].sort_values(ascending=False).unique().tolist() 
sort_order

[1.0, 0.99, 0.93, 0.92, 0.895, 0.71, 0.485, 0.46, 0.005]

In [218]:
# Multiple Population Proportion by 100 for better visualization
history_long_df['Population Proportion'] = history_long_df['Population Proportion'] * 100

In [219]:
# Number of discrete colors you want
num_colors = 9

# Generate a list of equally spaced values between 0 and 1
scale_values = np.linspace(0, 1, num_colors)

# Extract colors from the Viridis scale (in hexadecimal format)
viridis_colors_hex = px.colors.sample_colorscale('Viridis', scale_values, low=0.5, high=1)

viridis_colors_hex.reverse()

In [220]:
# Visualize the results 
# fig = px.line(history_long_df, x='Generation',
#               y='Population Proportion', 
#               line_group='Strategy', color='Cooperation Rating', color_discrete_sequence=viridis_colors_hex,
#               labels={'Population Proportion': 'Population Proportion', 'Generation': 'Generation', 'Cooperation Rating': 'Cooperation Rating', 'Strategy': 'Strategy'}, 
#               category_orders= {'Cooperation Rating': sort_order}, 
#               )

# fig.update_layout(autosize=False, width=550, height=500, margin=dict(l=5, r=10, b=10, t=50, pad=1), showlegend=True, title='Evolution of Strategies from Axelrod\'s First Tournament')
# fig.update_xaxes(categoryorder='array', categoryarray= history_long_df['Cooperation Rating'].sort_values(ascending=False).unique())

# fig.update_yaxes(range=[0, 15], ticksuffix="%")


# fig.show()

In [234]:
# Visualize the results 
fig = px.area(history_long_df, x='Generation',
              y='Population Proportion', 
              line_group='Strategy', color='Cooperation Rating', color_discrete_sequence=viridis_colors_hex,
              labels={'Population Proportion': 'Population Proportion', 'Generation': 'Generation', 'Cooperation Rating': 'Cooperation Rating', 'Strategy': 'Strategy'},
              category_orders= {'Cooperation Rating': sort_order}, 
              #animation_frame='Generation',
              #animation_group='Strategy',
              #  range_y=[0, 100], 
              #  range_x=[0, 100]
)

fig.update_layout(autosize=False, width=550, height=500, margin=dict(l=5, r=10, b=10, t=50, pad=1), showlegend=True, title='Evolution of Strategies from Axelrod\'s First Tournament')
fig.update_yaxes(range=[0, 100], ticksuffix="%")

fig.show()

In [235]:
fig.write_html("ess.html")